In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import torch
import torch.optim as optim

This is adapted from chapter 6 Deep learning with Pytorch book. We will look at how Pytorch nn module works.

## Lets get the data

In [ ]:
train = pd.read_csv('../input/celsius-to-fahrenheit/training.csv')
train.head()

In [ ]:
t_c = list(train['Celsius'])
t_u = list(train['Fahrenheit'])

t_c[:5], t_u[:5]


t_u and t_c were two 1D tensors of size B. In a linear model, no neural network, Thanks
to broadcasting, we could write our linear model as w * x + b, where w and b were
two scalar parameters.

This worked because we had a single input feature: if we had
two, we would need to add an extra dimension to turn that 1D tensor into a matrix
with samples in the rows and features in the columns.

That’s exactly what we need to do to switch to using nn.Linear. We reshape our B
inputs to B × Nin, where Nin is 1. That is easily done with unsqueeze:

In [ ]:
t_c = torch.tensor(t_c).unsqueeze(1)
t_u = torch.tensor(t_u).unsqueeze(1)

In [ ]:
t_c[:5], t_u[:5], t_u.shape, t_c.shape

## training and validation set

In [ ]:
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_indices, val_indices

In [ ]:
t_u_train = t_u[train_indices]
t_c_train = t_c[train_indices]

t_u_val = t_u[val_indices]
t_c_val = t_c[val_indices]


normalising the values

In [ ]:
t_un_train = 0.1 * t_u_train
t_un_val = 0.1 * t_u_val

In [ ]:
# t_cn_train = 0.1 * t_c_train
# t_cn_val = 0.1 * t_c_val

t_cn_train = t_c_train
t_cn_val = t_c_val

In [ ]:
t_cn_train, t_cn_val = t_cn_train.type(torch.FloatTensor), t_cn_val.type(torch.FloatTensor)


In [ ]:
type(t_cn_train), type(t_c_val)

In [ ]:
t_cn_train[:5], t_un_train[:5]

# The Pytorch nn module

 PyTorch has a whole submodule dedicated to neural networks, called torch.nn. It
contains the building blocks needed to create all sorts of neural network architectures. Those building blocks are called modules in PyTorch parlance (such building
blocks are often referred to as layers in other frameworks). A PyTorch module is a
Python class deriving from the nn.Module base class. A module can have one or more
Parameter instances as attributes, which are tensors whose values are optimized
during the training process (think w and b in our linear model). A module can also
have one or more submodules (subclasses of nn.Module) as attributes, and it will be
able to track their parameters as well.

In [ ]:
import torch.nn as nn

All PyTorch-provided subclasses of nn.Module have their __call__ method defined.
This allows us to instantiate an nn.Linear and call it as if it was a function, 

note: The submodules must be top-level attributes, not buried inside list or
dict instances! Otherwise, the optimizer will not be able to locate the submodules (and, hence, their parameters). For situations where your model
requires a list or dict of submodules, PyTorch provides nn.ModuleList and
nn.ModuleDict.

All PyTorch-provided subclasses of nn.Module have their __call__ method defined.
This allows us to instantiate an nn.Linear and call it as if it was a function, like so


In [ ]:
linear_model = nn.Linear(1,1)
# linear model accepts 3 : input size, output size and bias defaults to True
linear_model(t_un_val)

Aside : Calling an instance of nn.Module with a set of arguments ends up calling a method
named forward with the same arguments. The forward method is what executes the
forward computation, while __call__ does other rather important chores before and
after calling forward. So, it is technically possible to call forward directly, and it will
produce the same output as __call__, but this should not be done from user code:

`y = model(x)` correct

`y = model.forward(x)` wrong

In [ ]:
linear_model.weight

In [ ]:
linear_model.bias

In [ ]:
## calling the module with some input

linear_model(torch.ones(1))

Although PyTorch lets us get away with it, we don’t actually provide an input with the
right dimensionality. We have a model that takes one input and produces one output,
but PyTorch nn.Module and its subclasses are designed to do so on multiple samples at
the same time. To accommodate multiple samples, modules expect the zeroth dimension of the input to be the number of samples in the batch.

## Batching inputs

Any module in nn is written to produce outputs for a batch of multiple inputs at the
same time. Thus, assuming we need to run nn.Linear on 10 samples, we can create an
input tensor of size B × Nin, where B is the size of the batch and Nin is the number of
input features, and run it once through the model. For example:

In [ ]:
linear_model(torch.ones(10,1))

# since there is no loss and update the values are same

Our input is B × C × H × W with a batch size of 3 (say, images of a dog, a bird, and then a car), three channel dimensions (red, green, and blue),
and an unspecified number of pixels for height and width. As we can see, the output is a tensor of size B × Nout, where Nout is the number of output features: four, in
this case.

## Optimising batches

The reason we want to do this batching is multifaceted. One big motivation is to make
sure the computation we’re asking for is big enough to saturate the computing
resources we’re using to perform the computation. GPUs in particular are highly parallelized, so a single input on a small model will leave most of the computing units idle.
By providing batches of inputs, the calculation can be spread across the otherwise-idle
units, which means the batched results come back just as quickly as a single result
would. Another benefit is that some advanced models use statistical information from
the entire batch, and those statistics get better with larger batch sizes.

lets pass te linear model (parameters) to optimizer:

In [ ]:
linear_model = nn.Linear(1,1)

In [ ]:
optimizer = optim.SGD(
    linear_model.parameters(),
    lr=1e-2
)

Usually, it was our responsibility to create parameters and pass them as the first argument to optim.SGD. Now we can use the parameters method to ask any nn.Module for
a list of parameters owned by it or any of its submodules:

In [ ]:
list(linear_model.parameters())

This call recurses into submodules defined in the module’s init constructor and
returns a flat list of all parameters encountered, so that we can conveniently pass it to
the optimizer constructor as we did previously.

We can already figure out what happens in the training loop. The optimizer is provided with a list of tensors that were defined with requires_grad = True—all Parameters
are defined this way by definition, since they need to be optimized by gradient descent.

When training_loss.backward() is called, grad is accumulated on the leaf nodes of the
graph, which are precisely the parameters that were passed to the optimizer.

 At this point, the SGD optimizer has everything it needs. When optimizer.step()
is called, it will iterate through each Parameter and change it by an amount proportional to what is stored in its grad attribute. Pretty clean design.
 Let’s take a look a the training loop now:

In [ ]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val, t_c_train, t_c_val, output_freq):
    print("for t_u_train : ", t_u_train )
    print("for t_c_train : ", t_c_train)
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train)
        loss_train = loss_fn(t_p_train, t_c_train)
        
        t_p_val = model(t_u_val)
        loss_val = loss_fn(t_p_val, t_c_val )
        
        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()
        
    
        if epoch ==1 or epoch % output_freq == 0:
            print("predicted t_p_train : ", t_p_train)
            print("Epoch: ", epoch, " Training loss: ", loss_train.item(), " Val loss:", loss_val.item())

It hasn’t changed practically at all, except that now we don’t pass params explicitly to
model since the model itself holds its Parameters internally.
 There’s one last bit that we can leverage from torch.nn: the loss. Indeed, nn comes
with several common loss functions, among them nn.MSELoss (MSE stands for Mean
Square Error), which is exactly what we defined earlier as our loss_fn. Loss functions
in nn are still subclasses of nn.Module, so we will create an instance and call it as a
function. In our case, we get rid of the handwritten loss_fn and replace it.

### debug

my loss values were comingtoo high inf

In [ ]:
t_un_train

In [ ]:
t_cn_train

In [ ]:
torch.ones(5)

In [ ]:
linear_model = nn.Linear(1,1)
linear_model

In [ ]:
linear_model(torch.ones(1))

In [ ]:
a = [0,1,1]
b = [1,1,1]

a = torch.tensor(a).unsqueeze(1)
b = torch.tensor(b).unsqueeze(1)

a,b=a.type(torch.FloatTensor),b.type(torch.FloatTensor)

In [ ]:
# TRYING OUT LOSS FUNCTION
t_p_train = linear_model(a)
print(t_p_train)

test_loss= nn.MSELoss()
test_loss(t_p_train, b)

In [ ]:
linear_model= nn.Linear(1,1)
optimizer = optim.SGD(linear_model.parameters(), lr=1e-5)

training_loop(
    n_epochs = 3000,
    optimizer = optimizer,
    model = linear_model,
    loss_fn = nn.MSELoss(),
    t_u_train = t_un_train,
    t_u_val = t_un_val,
    t_c_train = t_cn_train,
    t_c_val = t_cn_val,
    output_freq = 1000
)

print()
print(linear_model.weight)
print(linear_model.bias)

Got an answer! 

Note if you are getting inf as the training loss, play with learning rate, it might be too high.

### Replacing the model

We are going to keep everything else fixed, including the loss function, and only redefine model. Let’s build the simplest possible neural network: a linear module, followed
by an activation function, feeding into another linear module.

In [ ]:
seq_model = nn.Sequential(
            nn.Linear(1,13),
    # 13 was chosen arbitrarly
            nn.Tanh(),
            nn.Linear(13,1)
)

seq_model

In [ ]:
seq_model.parameters()

In [ ]:
[param.shape for param in seq_model.parameters()]

These are the tensors that the optimizer will get. Again, after we call model.backward(),
all parameters are populated with their grad, and the optimizer then updates their values accordingly during the optimizer.step() call. Not that different from our previous
linear model, eh? After all, they’re both differentiable models that can be trained using
gradient descent.
 A few notes on parameters of nn.Modules. When inspecting parameters of a model
made up of several submodules, it is handy to be able to identify parameters by name.
There’s a method for that, called named_parameters:

In [ ]:
for name, param in seq_model.named_parameters():
    print(name, param.shape)

The name of each module in Sequential is just the ordinal with which the module
appears in the arguments. Interestingly, Sequential also accepts an OrderedDict,in
which we can name each module passed to Sequential:

In [ ]:
from collections import OrderedDict

seq_model = nn.Sequential(OrderedDict([
    ('hidden_linear', nn.Linear(1,8)),
    ('hidden_activation', nn.Tanh()),
    ('output_linear', nn.Linear(8,1))
]))

seq_model

This allows us to get more explanatory names for submodules:

In [ ]:
for name, param  in seq_model.named_parameters():
    print(name, param.shape)

This is more descriptive; but it does not give us more flexibility in the flow of data
through the network, which remains a purely sequential pass-through—the
nn.Sequential is very aptly named. We will see how to take full control of the processing of input data by subclassing nn.Module ourselves in chapter 8.
 We can also access a particular Parameter by using submodules as attributes:


In [ ]:
seq_model.output_linear.bias

This is useful for inspecting parameters or their gradients: for instance, to monitor
gradients during training, as we did at the beginning of this chapter. Say we want to
print out the gradients of weight of the linear portion of the hidden layer. We can run
the training loop for the new neural network model and then look at the resulting
gradients after the last epoch:


In [ ]:
optimizer = optim.SGD(seq_model.parameters(), lr = 1e-3)

training_loop(
    n_epochs=5000,
    optimizer=optimizer,
    model=seq_model,
    loss_fn=nn.MSELoss(),
    t_u_train=t_un_train,
    t_u_val=t_un_val,
    t_c_train=t_cn_train,
    t_c_val=t_cn_val,
    output_freq = 1000
)


In [ ]:
print('Output', seq_model(t_un_val))
print('Answer', t_c_val)
print('hidden', seq_model.hidden_linear.weight.grad)

### Comparing to a linear model

In [ ]:
from matplotlib import pyplot as plt

t_range = torch.arange(20.,90.).unsqueeze(1)

fig = plt.figure(dpi=600)

plt.xlabel("Fahrenheit")
plt.ylabel("Celsius")
plt.plot(t_u.numpy(), t_c.numpy(), 'o')
plt.plot(t_range.numpy(), seq_model(0.1 * t_range).detach().numpy(), 'c-')
plt.plot(t_u.numpy(), seq_model(0.1 * t_u).detach().numpy(), 'kx')

for linear model

In [ ]:
from matplotlib import pyplot as plt

t_range = torch.arange(20.,90.).unsqueeze(1)

fig = plt.figure(dpi=600)

plt.xlabel("Fahrenheit")
plt.ylabel("Celsius")
plt.plot(t_u.numpy(), t_c.numpy(), 'o')
plt.plot(t_range.numpy(), linear_model(0.1 * t_range).detach().numpy(), 'c-')
plt.plot(t_u.numpy(), linear_model(0.1 * t_u).detach().numpy(), 'kx')

. By now you should
have confidence in your understanding of what’s going on behind the scenes. Hopefully this taste of PyTorch has given you an appetite for mo

# Exercises

1. Experiment with the number of hidden neurons in our simple neural network
model, as well as the learning rate.

In [ ]:
# lets get data

import pandas as pd

train = pd.read_csv("../input/celsius-to-fahrenheit/training.csv")
train.head()

In [ ]:
t_c = list(train['Celsius'])
t_u = list(train['Fahrenheit'])

t_c[:5], t_u[:5]

In [ ]:
import torch

In [ ]:
t_c = torch.tensor(t_c).unsqueeze(1)
t_u = torch.tensor(t_u).unsqueeze(1)


In [ ]:
t_c[:5], t_u[:5]

In [ ]:
t_c.shape, t_u.shape

In [ ]:
t_c, t_u = t_c.type(torch.FloatTensor), t_u.type(torch.FloatTensor)


In [ ]:
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_indices, val_indices

In [ ]:
# validation set creation and normalization

t_un_train = t_u[train_indices] * 0.1
t_un_val = t_u[val_indices] * 0.1

t_c_train = t_c[train_indices] 
t_c_val = t_c[val_indices] 

In [ ]:
# lets create a model from scratch


seq_model = torch.nn.Sequential(
    torch.nn.Linear(1, 17),
    torch.nn.Tanh(),
    torch.nn.Linear(17,1)
)

In [ ]:
[param.shape for param in seq_model.parameters()]

In [ ]:
optimizer = torch.optim.SGD(seq_model.parameters(), lr=1e-3)

In [ ]:
t_un_train.shape, t_c_train.shape

In [ ]:
def training_loop(n_epochs=5000, optimizer=optimizer,model=seq_model,loss_fn=torch.nn.MSELoss(), train_t_u=t_un_train, val_t_u=t_un_val, train_t_c=t_c_train, val_t_c = t_c_val):
    for epoch in range(1, n_epochs +1):
        train_t_p = model(train_t_u)
        train_loss = loss_fn(train_t_p, train_t_c)
        
        with torch.no_grad():
            val_t_p = model(val_t_u)
            val_loss = loss_fn(val_t_p, val_t_c)
            assert val_loss.requires_grad == False
        
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        if (epoch %1000 == 0):
            print('training_loss: ', train_loss, 'validation_loss: ',val_loss)

In [ ]:
training_loop()